# Загрузка данных

In [1]:
!gdown 19WJRvdFZgV1hxmVl52HCKq0gieTQk_uY #flats_checks_raster.csv
!gdown 1tbxE-SZO5R37TiBscVZ2zbTGeTAIsUYk #таргет price трейн
!gdown 1ZI-JsZ-6QxtHJKnxj1Mggscxumodxm64 # таргет price тест

Downloading...
From (original): https://drive.google.com/uc?id=19WJRvdFZgV1hxmVl52HCKq0gieTQk_uY
From (redirected): https://drive.google.com/uc?id=19WJRvdFZgV1hxmVl52HCKq0gieTQk_uY&confirm=t&uuid=3e0f4049-6dce-4f21-a708-ee473a1bb53b
To: /content/flats_checks_raster.csv
100% 1.05G/1.05G [00:21<00:00, 48.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1tbxE-SZO5R37TiBscVZ2zbTGeTAIsUYk
To: /content/y_train_msk_merged_2.pkl
100% 788k/788k [00:00<00:00, 4.28MB/s]
Downloading...
From: https://drive.google.com/uc?id=1ZI-JsZ-6QxtHJKnxj1Mggscxumodxm64
To: /content/y_test_msk_merged_2.pkl
100% 140k/140k [00:00<00:00, 7.87MB/s]


# Предобработка датасетов. Выделение датасетов из единого

In [2]:
import pandas as pd
# 1. Загружаем и ставим индекс. Это требуется потому, что индексы слетели из-за сокращения датасета после добавления данных ФНС
df_mega_base = pd.read_csv('flats_checks_raster.csv', index_col='Unnamed: 0')

# 1.1. Загружаем y_train и y_test. Таргеты были отдельно
y_train = pd.read_pickle('y_train_msk_merged_2.pkl')
y_test = pd.read_pickle('y_test_msk_merged_2.pkl')

# 2. Объединяем их в один DataFrame
y_full = pd.concat([y_train, y_test])

# 3. Присоединяем таргеты к df_mega_base по индексу
df_mega_base = df_mega_base.join(y_full, how='left')
df_mega_base = df_mega_base.reset_index(drop=True)

# 2. Удаляем все столбцы с 'emb' в названии. Удаляем эмбеддинги Славы, т.к. датасет содержал эмбеддинги tabfpn, мне они не трубуются
df_mega_base = df_mega_base.loc[:, ~df_mega_base.columns.str.contains('emb')]

# 3. Удаляем столбцы
df_mega_base = df_mega_base.drop([
    'TruncatedAverageBill', 'MedianBill',
    'lat', 'lng', 'geometry', 'index_right',
    'coordinates', 'polygon'
], axis=1, errors='ignore')

In [3]:
# заполняем пропуски средним
for column in df_mega_base.columns:
    df_mega_base[column].fillna(df_mega_base[column].mean(), inplace=True)

# 4. Создаём df_check из столбцов ФНС. на нем будем валидироваться
df_check = df_mega_base[[
    'KktCount',
    'AverageBill',
    'CachePayPercent',
    'IntensityOfNumberBills',
    'RevenueIntensity',
    'ReceiptTotalCount',
    'price'
]].copy()

# 5. df_flats_rasters — это оставшиеся столбцы. Получаем датасет только из квартир + POI + растры
df_flats_rasters = df_mega_base.drop(columns=df_check.columns, errors='ignore')


# 6. Удаляем все столбцы, содержащие 'rast' в названии
df_flats = df_flats_rasters.loc[:, ~df_flats_rasters.columns.str.contains('rast')]

# 7. Делаем датасет, в котором есть только исходные данные по квартирам (ЦИАН)
POI = [
    'distance_to_center',
    'highways_count',
    'undergrounds_count',
    'railways_count',
    'time_to_metro',
    'pop_dense',
    'pop_work_dense',
    'pop_child_dense',
    'avg_age',
    'trade_per_pers',
    'pers_per_bed',
    'self_goods_sold',
    'other_goods_sold',
    'org_num',
    'entrep_num',
    'buildings_apartments',
    'buildings_service',
    'buildings_retail',
    'buildings_kindergarten',
    'buildings_school',
    'buildings_office',
    'buildings_construction',
    'buildings_commercial',
    'buildings_hospital',
    'buildings_university',
    'buildings_public',
    'buildings_industrial',
    'buildings_church',
    'education',
    'food_buy',
    'food_out',
    'health',
    'leisure',
    'religion',
    'services',
    'shopping',
    'transport',
    'mun_district',
]

df_flats_without_POI = df_flats.drop(columns=POI, errors='ignore')
df_flats_rasters_only = df_flats_rasters.drop(columns=POI, errors='ignore')


/tmp/ipython-input-3-535601612.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_mega_base[column].fillna(df_mega_base[column].mean(), inplace=True)


# GNN предсказание на датасете квартир + растровой гео-информации

In [4]:
!pip install torch_geometric
!pip install catboost
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from sklearn.neighbors import kneighbors_graph
from torch_geometric.nn import SAGEConv, BatchNorm
from sklearn.model_selection import train_test_split

# GNN
Датасет квартиры МСК

In [5]:
# === 1. Загрузка и подготовка данных ===
y = df_mega_base['price']
X = df_flats_without_POI.copy()

# Делим на train и test
df_train_flats, df_test_flats, y_train_price, y_test_price = train_test_split(
    X, y, test_size=0.2, random_state=999
)

# Масштабируем таргеты
target_scaler = StandardScaler()
y_train_scaled = target_scaler.fit_transform(y_train_price.values.reshape(-1, 1))
y_test_scaled = target_scaler.transform(y_test_price.values.reshape(-1, 1))

# Объединение
df_all = pd.concat([df_train_flats, df_test_flats], ignore_index=True)
y_all = np.vstack([y_train_scaled, y_test_scaled])
X_all = df_all.values.astype(np.float32)

# === 2. Построение графа и PyG Data ===
A = kneighbors_graph(X_all, n_neighbors=8, mode='connectivity', include_self=False)
edge_index = torch.tensor(np.array(A.nonzero()), dtype=torch.long)

In [6]:
# === NEW: определим устройство ===
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

data = Data(
    x=torch.tensor(X_all, dtype=torch.float),
    edge_index=edge_index,
    y=torch.tensor(y_all, dtype=torch.float)
)

# Маски
train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
test_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
train_mask[:len(df_train_flats)] = True
test_mask[len(df_train_flats):] = True
data.train_mask = train_mask
data.test_mask = test_mask

# Переносим data на GPU
data = data.to(device)

# === 3. GNN модель ===
class DeepGNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, dropout=0.3):
        super(DeepGNN, self).__init__()
        self.dropout = dropout
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)
        self.conv3 = SAGEConv(hidden_channels, hidden_channels)
        self.mlp = torch.nn.Sequential(
            torch.nn.Linear(hidden_channels, hidden_channels // 2),
            torch.nn.ReLU(),
            torch.nn.Dropout(dropout),
            torch.nn.Linear(hidden_channels // 2, out_channels)
        )

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)

        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)

        x = self.conv3(x, edge_index)
        x = F.relu(x)

        return self.mlp(x)

    def get_embeddings(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        return x

# Инициализация модели и оптимизатора
model = DeepGNN(
    in_channels=X_all.shape[1],
    hidden_channels=128,
    out_channels=1
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
loss_fn = torch.nn.MSELoss()

In [7]:
model.train()
for epoch in range(500):
    optimizer.zero_grad()
    out = model(data.x, data.edge_index).squeeze()
    loss = loss_fn(out[data.train_mask], data.y[data.train_mask].squeeze())
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item():.4f}')

# === 5. Оценка модели (RMSE на тесте) ===
model.eval()
with torch.no_grad():
    preds_scaled = model(data.x, data.edge_index).squeeze()
    preds_test_scaled = preds_scaled[data.test_mask].cpu().numpy()
    y_test_scaled_true = data.y[data.test_mask].squeeze().cpu().numpy()

    # Обратное масштабирование
    preds_test = target_scaler.inverse_transform(preds_test_scaled.reshape(-1, 1))
    y_test_true = target_scaler.inverse_transform(y_test_scaled_true.reshape(-1, 1))

    # Вычисляем RMSE
    rmse = mean_squared_error(y_test_true, preds_test) ** 0.5
    print(f'\nTest RMSE: {rmse:.2f}')

Epoch 0, Loss: 48.4949
Epoch 10, Loss: 17.3288
Epoch 20, Loss: 10.1148
Epoch 30, Loss: 6.6460
Epoch 40, Loss: 4.6447
Epoch 50, Loss: 3.6701
Epoch 60, Loss: 2.9539
Epoch 70, Loss: 2.5227
Epoch 80, Loss: 2.1545


KeyboardInterrupt: 

# GNN
Датасет квартиры МСК + POI

In [ ]:
# === 1. Загрузка и подготовка данных ===
y = df_mega_base['price']
X = df_flats.copy()

# Делим на train и test
df_train_flats, df_test_flats, y_train_price, y_test_price = train_test_split(
    X, y, test_size=0.2, random_state=999
)

# Масштабируем таргеты
target_scaler = StandardScaler()
y_train_scaled = target_scaler.fit_transform(y_train_price.values.reshape(-1, 1))
y_test_scaled = target_scaler.transform(y_test_price.values.reshape(-1, 1))

# Объединение
df_all = pd.concat([df_train_flats, df_test_flats], ignore_index=True)
y_all = np.vstack([y_train_scaled, y_test_scaled])
X_all = df_all.values.astype(np.float32)

# === 2. Построение графа и PyG Data ===
A = kneighbors_graph(X_all, n_neighbors=8, mode='connectivity', include_self=False)
edge_index = torch.tensor(np.array(A.nonzero()), dtype=torch.long)

In [ ]:
# === NEW: определим устройство ===
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

data = Data(
    x=torch.tensor(X_all, dtype=torch.float),
    edge_index=edge_index,
    y=torch.tensor(y_all, dtype=torch.float)
)

# Маски
train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
test_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
train_mask[:len(df_train_flats)] = True
test_mask[len(df_train_flats):] = True
data.train_mask = train_mask
data.test_mask = test_mask

# Переносим data на GPU
data = data.to(device)

# === 3. GNN модель ===
class DeepGNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, dropout=0.3):
        super(DeepGNN, self).__init__()
        self.dropout = dropout
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)
        self.conv3 = SAGEConv(hidden_channels, hidden_channels)
        self.mlp = torch.nn.Sequential(
            torch.nn.Linear(hidden_channels, hidden_channels // 2),
            torch.nn.ReLU(),
            torch.nn.Dropout(dropout),
            torch.nn.Linear(hidden_channels // 2, out_channels)
        )

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)

        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)

        x = self.conv3(x, edge_index)
        x = F.relu(x)

        return self.mlp(x)

    def get_embeddings(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        return x

# Инициализация модели и оптимизатора
model = DeepGNN(
    in_channels=X_all.shape[1],
    hidden_channels=128,
    out_channels=1
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
loss_fn = torch.nn.MSELoss()

In [ ]:
model.train()
for epoch in range(500):
    optimizer.zero_grad()
    out = model(data.x, data.edge_index).squeeze()
    loss = loss_fn(out[data.train_mask], data.y[data.train_mask].squeeze())
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item():.4f}')

# === 5. Оценка модели (RMSE на тесте) ===
model.eval()
with torch.no_grad():
    preds_scaled = model(data.x, data.edge_index).squeeze()
    preds_test_scaled = preds_scaled[data.test_mask].cpu().numpy()
    y_test_scaled_true = data.y[data.test_mask].squeeze().cpu().numpy()

    # Обратное масштабирование
    preds_test = target_scaler.inverse_transform(preds_test_scaled.reshape(-1, 1))
    y_test_true = target_scaler.inverse_transform(y_test_scaled_true.reshape(-1, 1))

    # Вычисляем RMSE
    rmse = mean_squared_error(y_test_true, preds_test) ** 0.5
    print(f'\nTest RMSE: {rmse:.2f}')

# GNN
Датасет квартиры МСК + растровые данные

In [ ]:
# === 1. Загрузка и подготовка данных ===
y = df_mega_base['price']
X = df_flats_rasters

# Делим на train и test
df_train_flats, df_test_flats, y_train_price, y_test_price = train_test_split(
    X, y, test_size=0.2, random_state=999
)

# Масштабируем таргеты
target_scaler = StandardScaler()
y_train_scaled = target_scaler.fit_transform(y_train_price.values.reshape(-1, 1))
y_test_scaled = target_scaler.transform(y_test_price.values.reshape(-1, 1))

# Объединение
df_all = pd.concat([df_train_flats, df_test_flats], ignore_index=True)
y_all = np.vstack([y_train_scaled, y_test_scaled])
X_all = df_all.values.astype(np.float32)

# === 2. Построение графа и PyG Data ===
A = kneighbors_graph(X_all, n_neighbors=8, mode='connectivity', include_self=False)
edge_index = torch.tensor(np.array(A.nonzero()), dtype=torch.long)

In [ ]:
# === NEW: определим устройство ===
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

data = Data(
    x=torch.tensor(X_all, dtype=torch.float),
    edge_index=edge_index,
    y=torch.tensor(y_all, dtype=torch.float)
)

# Маски
train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
test_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
train_mask[:len(df_train_flats)] = True
test_mask[len(df_train_flats):] = True
data.train_mask = train_mask
data.test_mask = test_mask

# Переносим data на GPU
data = data.to(device)

# === 3. GNN модель ===
class DeepGNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, dropout=0.3):
        super(DeepGNN, self).__init__()
        self.dropout = dropout
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)
        self.conv3 = SAGEConv(hidden_channels, hidden_channels)
        self.mlp = torch.nn.Sequential(
            torch.nn.Linear(hidden_channels, hidden_channels // 2),
            torch.nn.ReLU(),
            torch.nn.Dropout(dropout),
            torch.nn.Linear(hidden_channels // 2, out_channels)
        )

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)

        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)

        x = self.conv3(x, edge_index)
        x = F.relu(x)

        return self.mlp(x)

    def get_embeddings(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        return x

# Инициализация модели и оптимизатора
model = DeepGNN(
    in_channels=X_all.shape[1],
    hidden_channels=128,
    out_channels=1
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
loss_fn = torch.nn.MSELoss()

In [ ]:
model.train()
for epoch in range(500):
    optimizer.zero_grad()
    out = model(data.x, data.edge_index).squeeze()
    loss = loss_fn(out[data.train_mask], data.y[data.train_mask].squeeze())
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item():.4f}')

# === 5. Оценка модели (RMSE на тесте) ===
model.eval()
with torch.no_grad():
    preds_scaled = model(data.x, data.edge_index).squeeze()
    preds_test_scaled = preds_scaled[data.test_mask].cpu().numpy()
    y_test_scaled_true = data.y[data.test_mask].squeeze().cpu().numpy()

    # Обратное масштабирование
    preds_test = target_scaler.inverse_transform(preds_test_scaled.reshape(-1, 1))
    y_test_true = target_scaler.inverse_transform(y_test_scaled_true.reshape(-1, 1))

    # Вычисляем RMSE
    rmse = mean_squared_error(y_test_true, preds_test) ** 0.5
    print(f'\nTest RMSE: {rmse:.2f}')

# GNN
Датасет квартиры МСК + POI + растровые данные

In [ ]:
# === 1. Загрузка и подготовка данных ===
y = df_mega_base['price']
X = df_flats_rasters.copy()

# Делим на train и test
df_train_flats, df_test_flats, y_train_price, y_test_price = train_test_split(
    X, y, test_size=0.2, random_state=999
)

# Масштабируем таргеты
target_scaler = StandardScaler()
y_train_scaled = target_scaler.fit_transform(y_train_price.values.reshape(-1, 1))
y_test_scaled = target_scaler.transform(y_test_price.values.reshape(-1, 1))

# Объединение
df_all = pd.concat([df_train_flats, df_test_flats], ignore_index=True)
y_all = np.vstack([y_train_scaled, y_test_scaled])
X_all = df_all.values.astype(np.float32)

# === 2. Построение графа и PyG Data ===
A = kneighbors_graph(X_all, n_neighbors=8, mode='connectivity', include_self=False)
edge_index = torch.tensor(np.array(A.nonzero()), dtype=torch.long)

In [ ]:
# === NEW: определим устройство ===
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

data = Data(
    x=torch.tensor(X_all, dtype=torch.float),
    edge_index=edge_index,
    y=torch.tensor(y_all, dtype=torch.float)
)

# Маски
train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
test_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
train_mask[:len(df_train_flats)] = True
test_mask[len(df_train_flats):] = True
data.train_mask = train_mask
data.test_mask = test_mask

# Переносим data на GPU
data = data.to(device)

# === 3. GNN модель ===
class DeepGNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, dropout=0.3):
        super(DeepGNN, self).__init__()
        self.dropout = dropout
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)
        self.conv3 = SAGEConv(hidden_channels, hidden_channels)
        self.mlp = torch.nn.Sequential(
            torch.nn.Linear(hidden_channels, hidden_channels // 2),
            torch.nn.ReLU(),
            torch.nn.Dropout(dropout),
            torch.nn.Linear(hidden_channels // 2, out_channels)
        )

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)

        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)

        x = self.conv3(x, edge_index)
        x = F.relu(x)

        return self.mlp(x)

    def get_embeddings(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        return x

# Инициализация модели и оптимизатора
model = DeepGNN(
    in_channels=X_all.shape[1],
    hidden_channels=128,
    out_channels=1
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.MSELoss()

In [ ]:
model.train()
for epoch in range(100):
    optimizer.zero_grad()
    out = model(data.x, data.edge_index).squeeze()
    loss = loss_fn(out[data.train_mask], data.y[data.train_mask].squeeze())
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item():.4f}')

# === 5. Оценка модели (RMSE на тесте) ===
model.eval()
with torch.no_grad():
    preds_scaled = model(data.x, data.edge_index).squeeze()
    preds_test_scaled = preds_scaled[data.test_mask].cpu().numpy()
    y_test_scaled_true = data.y[data.test_mask].squeeze().cpu().numpy()

    # Обратное масштабирование
    preds_test = target_scaler.inverse_transform(preds_test_scaled.reshape(-1, 1))
    y_test_true = target_scaler.inverse_transform(y_test_scaled_true.reshape(-1, 1))

    # Вычисляем RMSE
    rmse = mean_squared_error(y_test_true, preds_test) ** 0.5
    print(f'\nTest RMSE: {rmse:.2f}')

Это лучший результат, извлечем его эмбеддинги для дальнейшего использования

In [ ]:
# === 5. Извлечение эмбеддингов ===
model.eval()
with torch.no_grad():
    embeddings = model.get_embeddings(data.x, data.edge_index).cpu().numpy()

In [ ]:
# 1. Создаем DataFrame с эмбеддингами и правильными индексами из X_all
embeddings = pd.DataFrame(embeddings, index=df_mega_base.index)
embeddings.to_csv('embeddings.csv')

In [8]:
#подгружаем эмбеддинги, чтобы не пересчитывать
!gdown 1pZIcstb6kL90U8NfTpDKd7TOHr2FtEFC
embeddings = pd.read_csv('embeddings.csv')

Downloading...
From: https://drive.google.com/uc?id=1pZIcstb6kL90U8NfTpDKd7TOHr2FtEFC
To: /content/embeddings.csv
100% 46.3M/46.3M [00:00<00:00, 90.6MB/s]


# Валидация. Датасет с фискальными данными

## Предсказание catboost на сыром датасете

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# === 1. Берем df_check и определяем X и y ===
target_col = 'AverageBill'
X = df_check.drop(columns=[target_col])
y = df_check[target_col]

# === 2. Делим на train/test ===
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=999
)

# === 3. Масштабируем признаки по train ===
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# === 4. Обучение CatBoost ===
model = CatBoostRegressor(
    iterations=500,
    learning_rate=0.1,
    depth=6,
    verbose=0,
    random_state=999
)
model.fit(X_train, y_train)

# === 5. Предсказания и оценка RMSE ===
y_pred = model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred) ** 0.5
print(f'RMSE: {rmse:.2f}')


RMSE: 120.35


## Предсказание catboost на датасете с добавленными эмбеддингами из GNN

In [10]:
# 1. Создаем DataFrame с эмбеддингами и правильными индексами из X_all
emb_df = pd.DataFrame(embeddings, index=df_mega_base.index)
emb_df.columns = [f'emb_{i}' for i in range(embeddings.shape[1])]

# 2. Присоединяем эмбеддинги по индексу
existing_cols = set(df_check.columns)

# 3. Отбираем только те столбцы из emb_df, которых нет в df_check
emb_to_add = emb_df[[col for col in emb_df.columns if col not in existing_cols]]

# 4. Присоединяем по индексу
df_check_emb = df_check.join(emb_to_add, how='left')

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# === 1. Берем df_check и определяем X и y ===
target_col = 'AverageBill'
X = df_check_emb.drop(columns=[target_col])
y = df_check_emb[target_col]

# === 2. Делим на train/test ===
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=999
)

# === 3. Масштабируем признаки по train ===
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# === 4. Обучение CatBoost ===
model = CatBoostRegressor(
    iterations=500,
    learning_rate=0.1,
    depth=6,
    verbose=0,
    random_state=999
)
model.fit(X_train, y_train)

# === 5. Предсказания и оценка RMSE ===
y_pred = model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred) ** 0.5
print(f'RMSE: {rmse:.2f}')


RMSE: 130.98


На датасете с фискальной информацией полученные GNN на квартирах эмбеддинги не дали улучшения метрики, а наоборот - ухудшили результаты. Видимо, они внесли шум, либо датасет квартир был недостаточно большим